## Cost Efficiency Goals Analysis (Flag 81)

### Dataset Overview
This dataset includes 500 simulated entries from the ServiceNow `sn_gf_goal` table, detailing various aspects of organizational goals. Key attributes include goal status, responsible parties, associated departments, start and end dates, and detailed descriptions. The dataset also contains metrics such as priority ranking, percentage completion, and target achievement rate. It primarily focuses on tracking the progress and management of both departmental and individual goals, offering insights into the effectiveness and alignment of these goals with broader organizational strategies. Additionally, the dataset logs updates to each goal, providing a record of modifications and the identities of those making these changes.

### Your Objective
**Objective**: Analyze the factors (whether time-based or categorical) that influence goal completion rates and evaluate their impact on overall departmental performance.

**Role**: Enterprise Performance Analyst

**Challenge Level**: 4 out of 5. The task requires a deep understanding of complex data patterns and advanced analytical skills to interpret the information and extract actionable insights.

**Category**: Strategic Goal Management

### Import Necessary Libraries
This cell imports all necessary libraries required for the analysis. This includes libraries for data manipulation, data visualization, and any specific utilities needed for the tasks. 

In [1]:
import argparse
import pandas as pd
import json
import requests
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from pandas import date_range

## Load Dataset
This cell loads the goals dataset used in the analysis. The dataset is assumed to be stored in a CSV file and is loaded into a DataFrame. This step includes reading the data from a file path and possibly performing initial observations such as viewing the first few rows to ensure it has loaded correctly.


In [2]:
dataset_path = "csvs/flag-81.csv"
goal_data = pd.read_csv(dataset_path)
df = pd.read_csv(dataset_path)
goal_data.head()


,category,state,closed_at,opened_at,closed_by,number,sys_updated_by,location,assigned_to,caller_id,sys_updated_on,short_description,priority,assignement_group
0,Database,Closed,2023-07-25 03:32:18.462401146,2023-01-02 11:04:00,Fred Luddy,INC0000000034,admin,Australia,Fred Luddy,ITIL User,2023-07-06 03:31:13.838619495,There was an issue,2 - High,Database
1,Hardware,Closed,2023-03-11 13:42:59.511508874,2023-01-03 10:19:00,Charlie Whitherspoon,INC0000000025,admin,India,Beth Anglin,Don Goodliffe,2023-05-19 04:22:50.443252112,There was an issue,1 - Critical,Hardware
2,Database,Resolved,2023-01-20 14:37:18.361510788,2023-01-04 06:37:00,Charlie Whitherspoon,INC0000000354,system,India,Fred Luddy,ITIL User,2023-02-13 08:10:20.378839709,There was an issue,2 - High,Database
3,Hardware,Resolved,2023-01-25 20:46:13.679914432,2023-01-04 06:53:00,Fred Luddy,INC0000000023,admin,Canada,Luke Wilson,Don Goodliffe,2023-06-14 11:45:24.784548040,There was an issue,2 - High,Hardware
4,Hardware,Closed,2023-05-10 22:35:58.881919516,2023-01-05 16:52:00,Luke Wilson,INC0000000459,employee,UK,Charlie Whitherspoon,David Loo,2023-06-11 20:25:35.094482408,There was an issue,2 - High,Hardware



### **Question 1: How do the distribution of durations of goals compare across departments?**

#### Plot comparison of goal durations across departments

This cell plots a comprehensive look at how the durations of goals differ across various departments within the organization. Using a box plot, we compare the spread and central tendency of goal durations, marked by the median and mean durations for each department. This analysis is for identifying departments that may require process adjustments to align more closely with organizational goal completion times. Each department's performance is annotated directly on the plot, displaying both median and mean durations, which helps in quickly assessing and comparing departmental efficiency in achieving goals.

In [3]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# import pandas as pd
# import numpy as np

# # Assuming 'goal_data' is preloaded and contains 'Cost Reduction' category
# goal_data['end_date'] = pd.to_datetime(goal_data['end_date'])
# goal_data["start_date"] = pd.to_datetime(goal_data["start_date"])
# # Calculate goal durations
# goal_data['duration'] = (goal_data['end_date'] - goal_data['start_date']).dt.days

# # Plotting
# plt.figure(figsize=(12, 8))
# box_plot = sns.boxplot(x='department', y='duration', data=goal_data, palette="Set3")
# plt.title('Comparison of Goal Durations by Department')
# plt.xlabel('Department')
# plt.ylabel('Goal Duration (days)')
# plt.grid(True)

# # Calculate median and mean for annotations
# medians = goal_data.groupby(['department'])['duration'].median()
# means = goal_data.groupby(['department'])['duration'].mean()

# # Iterate over the departments to place the text annotations for median and mean
# for xtick in box_plot.get_xticks():
#     box_plot.text(xtick, medians[xtick] + 1, 'Median: {:.1f}'.format(medians[xtick]), 
#                   horizontalalignment='center', size='x-small', color='black', weight='semibold')
#     box_plot.text(xtick, means[xtick] + 1, 'Mean: {:.1f}'.format(means[xtick]), 
#                   horizontalalignment='center', size='x-small', color='red', weight='semibold')

# plt.show()

print("N/A")

N/A


#### Generate JSON Description for the Insight

In [4]:
{
	"data_type": "analytical",
	"insight": "There was no column end_date to conduct any analysis",
	"insight_value": {
	},
	"plot": {
    	"description": "The graph could not be generated due to missing data",
	},
	"question": "How do the durations of 'Cost Reduction' goals in the Finance department compare to those in other departments?",
	"actionable_insight": "No actionable insight could be generated due to missing data"
}

{'data_type': 'analytical',
 'insight': 'There was no column end_date to conduct any analysis',
 'insight_value': {},
 'plot': {'description': 'The graph could not be generated due to missing data'},
 'question': "How do the durations of 'Cost Reduction' goals in the Finance department compare to those in other departments?",
 'actionable_insight': 'No actionable insight could be generated due to missing data'}

### **Question 2:** What is completion rates by different quarters?

#### Plot goal completion rates by quarter

This box plot visualizes the completion rates of goals across different quarters, providing insights into the seasonal trends in goal achievement. By segmenting the data into quarters, we can identify any patterns or variations in goal completion rates over the year. This analysis helps in understanding the impact of seasonality on goal management and can guide resource allocation and goal setting strategies based on historical trends.

In [5]:
# # Convert start_date to datetime format
# df['start_date'] = pd.to_datetime(df['start_date'])

# # Extract the month and quarter from the start_date
# df['month'] = df['start_date'].dt.month
# df['quarter'] = df['start_date'].dt.quarter

# # Visualize the trend of percent_complete by quarter
# plt.figure(figsize=(12, 6))
# sns.boxplot(x='quarter', y='percent_complete', data=df)
# plt.title('Percent Complete by Quarter')
# plt.xlabel('Quarter')
# plt.ylabel('Percent Complete')
# plt.grid(True, axis='y', linestyle='--', alpha=0.7)
# plt.show()

print("N/A")

N/A


In [6]:
{
	"data_type": "Diagnostic",
	"insight": "There was no column start_date to conduct any analysis",
	"insight_value": {
	},
	"plot": {
    	"description": "The graph could not be generated due to missing data",
	},
	"question": "How does the time of year (quarter) impact the completion rate of tasks?",
	"actionable_insight": "No actionable insight could be generated due to missing data"
}

{'data_type': 'Diagnostic',
 'insight': 'There was no column start_date to conduct any analysis',
 'insight_value': {},
 'plot': {'description': 'The graph could not be generated due to missing data'},
 'question': 'How does the time of year (quarter) impact the completion rate of tasks?',
 'actionable_insight': 'No actionable insight could be generated due to missing data'}

### **Question 3:** What is distribution of Goal types in Finance department?

#### Plot goal category distribution within the Finance department

This pie chart illustrates the proportional distribution of different goal categories within the Finance department, highlighting the dominant categories and their respective shares. This visualization is crucial for understanding the strategic focus areas within the department, based on the types of goals they prioritize.

In [7]:
# import matplotlib.pyplot as plt

# # Filter data for the Finance department
# finance_goals = goal_data[goal_data['department'] == 'Finance']

# # Count the occurrence of each category in the Finance department
# category_counts = finance_goals['category'].value_counts()

# # Create a pie chart
# plt.figure(figsize=(10, 7))
# plt.pie(category_counts, labels=category_counts.index, autopct='%1.1f%%', startangle=140)
# plt.title('Distribution of Goal Categories in Finance Department')
# plt.show()

print("N/A")

N/A


#### Generate JSON Description for the Insight

In [8]:
{
	"data_type": "diagnostic",
	"insight": "There was no column department to conduct any analysis",
	"insight_value": {
	},
	"plot": {
    	"description": "The graph could not be generated due to missing data",
	},
	"question": "What is the distribution of Goal types and categories in the Finance department?",
	"actionable_insight": "No actionable insight could be generated due to missing data"
}


{'data_type': 'diagnostic',
 'insight': 'There was no column department to conduct any analysis',
 'insight_value': {},
 'plot': {'description': 'The graph could not be generated due to missing data'},
 'question': 'What is the distribution of Goal types and categories in the Finance department?',
 'actionable_insight': 'No actionable insight could be generated due to missing data'}

### **Question 4:** What is distribution of Goal priorities in Finance department?

#### Plot priority distribution within the Finance department

This pie chart provides a clear visual representation of the distribution of goal priorities within the Finance department, emphasizing the prevalence of each priority level and aiding in the assessment of focus and urgency applied to different objectives.

In [9]:
# import matplotlib.pyplot as plt

# # Filter data for the Finance department
# finance_goals = goal_data[goal_data['department'] == 'Finance']

# # Count the occurrence of each category in the Finance department
# category_counts = finance_goals['priority'].value_counts()

# # Create a pie chart
# plt.figure(figsize=(10, 7))
# plt.pie(category_counts, labels=category_counts.index, autopct='%1.1f%%', startangle=140)
# plt.title('Distribution of Goal priorities in Finance Department')
# plt.show()

print("N/A")

N/A


#### Generate JSON Description for the Insight

In [10]:
{
	"data_type": "diagnostic",
	"insight": "There was no column department to conduct any analysis",
	"insight_value": {
	},
	"plot": {
    	"description": "The graph could not be generated due to missing data",
	},
	"question": "What is the distribution of Goal priorities in the Finance department?",
	"actionable_insight": "No actionable insight could be generated due to missing data"
}


{'data_type': 'diagnostic',
 'insight': 'There was no column department to conduct any analysis',
 'insight_value': {},
 'plot': {'description': 'The graph could not be generated due to missing data'},
 'question': 'What is the distribution of Goal priorities in the Finance department?',
 'actionable_insight': 'No actionable insight could be generated due to missing data'}

### Summary of Findings (Flag 81)

1. **Comparison of Goal Durations in Finance vs. Other Departments**: The absence of the `end_date` column prevents the analysis of how the durations of 'Cost Reduction' goals in the Finance department compare to those in other departments, limiting insights into departmental efficiency.

2. **Impact of Time of Year on Task Completion Rates**: Without the `start_date` column, it is impossible to analyze how the completion rates of tasks vary by quarter, preventing an understanding of any seasonal trends in task performance.

3. **Distribution of Goal Types and Categories in Finance**: The lack of the `department` column restricts the ability to analyze the distribution of goal types and categories specifically within the Finance department, hindering targeted performance analysis.

4. **Distribution of Goal Priorities in Finance**: The absence of the `department` column also limits the ability to assess the distribution of goal priorities in the Finance department, making it difficult to identify focus areas and potential improvement strategies.